# Project (Section 4)

This was the code we ended up with in the last section's project:

In [1]:
from datetime import date
from enum import Enum
from pydantic import BaseModel, ConfigDict


class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"


class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
    )
    
    manufacturer: str
    series_name: str
    type_: AutomobileType
    is_electric: bool = False
    manufactured_date: date
    base_msrp_usd: float
    vin: str
    number_of_doors: int = 4
    registration_country: str | None = None
    license_plate: str | None = None

Modify your `Automobile` model to implement the following:

- auto generate camel case aliases
- the field `type_` in our model is provided as `type` in source data, and should also serialize to `type`.
- the data we receive contains the following field names that need to map to our own model field names - but we still want our camelized field names to be used for serialization. Account for that (without renaming the field names):
    - `number_of_doors` is provided as `doors`
    - `manufactured_date` is provided as `completionDate`
- The field `base_msrp_usd` is provided as `msrpUSD`, and we want the serialization name to be `baseMSRPUSD`
- we want the JSON serialized output of `manufactured_date` to be this pattern: `YYYY/MM/DD` (e.g. `2020/01/01`), but serializing to a Python dict should remain as a `date` object.

In [9]:
from datetime import date
from enum import Enum
from pydantic import BaseModel, ConfigDict, Field, AliasChoices, field_serializer
from pydantic.alias_generators import to_camel


class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"


class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel
    )
    
    manufacturer: str
    series_name: str
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = False
    manufactured_date: date = Field(validation_alias="completionDate")
    base_msrp_usd: float = Field(validation_alias="msrpUSD", serialization_alias="baseMSRPUSD")
    vin: str
    number_of_doors: int = Field(default=4, validation_alias="doors")
    registration_country: str | None = None
    license_plate: str | None = None

    @field_serializer("manufactured_date", when_used="json-unless-none")
    def searialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")

The following source data:

In [10]:
data_json = '''
{
    "manufacturer": "BMW",
    "seriesName": "M4",
    "type": "Convertible",
    "isElectric": false,
    "completionDate": "2023-01-01",
    "msrpUSD": 93300,
    "vin": "1234567890",
    "doors": 2,
    "registrationCountry": "France",
    "licensePlate": "AAA-BBB"
}
'''

should deserialaze to these:

In [11]:
expected_serialized_dict = {
    'manufacturer': 'BMW',
    'series_name': 'M4',
    'type_': AutomobileType.convertible,
    'is_electric': False,
    'manufactured_date': date(2023, 1, 1),
    'base_msrp_usd': 93300.0,
    'vin': '1234567890',
    'number_of_doors': 2,
    'registration_country': 'France',
    'license_plate': 'AAA-BBB'
}

In [12]:
expected_serialized_dict_by_alias = {
    'manufacturer': 'BMW',
    'seriesName': 'M4',
    'type': AutomobileType.convertible,
    'isElectric': False,
    'manufacturedDate': date(2023, 1, 1),
    'baseMSRPUSD': 93300.0,
    'vin': '1234567890',
    'numberOfDoors': 2,
    'registrationCountry': 'France',
    'licensePlate': 'AAA-BBB'
}

In [13]:
expected_serialized_json_by_alias = (
    '{"manufacturer":"BMW","seriesName":"M4","type":"Convertible",'
    '"isElectric":false,"manufacturedDate":"2023/01/01","baseMSRPUSD":93300.0,'
    '"vin":"1234567890","numberOfDoors":2,"registrationCountry":"France",'
    '"licensePlate":"AAA-BBB"}'
)

In [15]:
car = Automobile.model_validate_json(data_json)
car

Automobile(manufacturer='BMW', series_name='M4', type_=<AutomobileType.convertible: 'Convertible'>, is_electric=False, manufactured_date=datetime.date(2023, 1, 1), base_msrp_usd=93300.0, vin='1234567890', number_of_doors=2, registration_country='France', license_plate='AAA-BBB')

In [16]:
assert car.model_dump() == expected_serialized_dict

In [17]:
assert car.model_dump(by_alias=True) == expected_serialized_dict_by_alias

In [18]:
assert car.model_dump_json(by_alias=True) == expected_serialized_json_by_alias